In [40]:
# import appropriate libraries
import pandas as pd
import numpy as np
import json, requests, urllib, io

In [41]:
# pull raw data
path_2015_2018_pitch = "Data/Raw/2015-2018_pitches.csv"
old_pitch = pd.read_csv(path_2015_2018_pitch)
path_2015_2018_atbats = "Data/Raw/2015-2018_atbats.csv"
old_atbat = pd.read_csv(path_2015_2018_atbats)



In [42]:
# merge raw data w/ at bat to get pitcher/batter info
old_pitch_merged = old_pitch.merge(old_atbat, left_on="ab_id", right_on="ab_id")

In [44]:
# fitler relevant columns, rename columns
keep_filter = ['px', 'pz', 'start_speed', 'zone', 'code', 'pitch_type', 
               'ab_id', 'b_count', 's_count','batter_id', 'event','pitcher_id','type']

old_pitch_merged_to_concat = old_pitch_merged[keep_filter]
total_pitch = old_pitch_merged_to_concat

In [45]:
# consider "SI" to be "FT"
total_pitch["pitch_type"].replace({"SI":"FT"}, inplace=True)

/project/engineering/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [46]:
# filter by 6 most popular pitch types
pitches = ["FF", "FT", "CU", "CH", "FC", "SL"]
filt = total_pitch['pitch_type'].isin(pitches)
total_pitch = total_pitch.loc[filt]

In [47]:
# rename pz to py
pitch = total_pitch
pitch.rename(columns={'pz':'py'}, inplace=True)

/project/engineering/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [49]:
#make and generate swing column
pitch['swing'] = -1
pitch['result'] = ""

# All possible results of a pitch if the batter swings
strike, out, hit, foul,  = "Strike", "Out", "Hit", "Foul"

# All possible results of a pitch if the batter takes
ball, take_strike = "Ball", "Take Strike"

# Type (More General than Code)
# X - In Play (This contains all X, D, E, BUT NOT H)
# B - Ball (swing or take the pitch was a ball)
# S - Strike (swing or take the pitch was a ball)

# Swinging Row Codes:
# F - Foul
# S - Swinging Strike
# T - Foul Tip (treat as swinging strike)
# L - Foul Bunt (treat as swinging strike)
# W - Swinging Strike Blocked (ball in dirt but batter swings)
# M - Missed Bunt (treat as swinging strike)
# Q - Swinging Pitchout
# R - Fouled Pitchout

# X - In play, out(s)
# D - In play, no out
# E - In play, runs
# H - Hit by Pitch 

# Taking Row Codes:
# B - Ball
# *B - Ball in Dirt
# I - Intentional Ball
# P - Pitchout (intentional ball trying to tag base stealer)
# C - Called Strike

for i, row in pitch.iterrows():
    if i % 10000 == 0: print(i)
    # batter swings and ball is NOT in play
    if row.code == 'S' or row.code == 'T' or row.code == 'L' or row.code == 'W' or row.code == 'M' or row.code == 'Q':
      pitch.at[i, 'swing'] = 1
      pitch.at[i, 'result'] = strike

    # batter swings and ball is NOT in play
    elif row.code == 'F' or row.code == 'R':
      pitch.at[i, 'swing'] = 1
      pitch.at[i, 'result'] = foul

    # batter takes and its a ball
    elif row.code == 'B' or row.code == '*B':
      pitch.at[i, 'swing'] = 0
      pitch.at[i, 'result'] = ball

    # pitcher intentionally throws a ball to the batter
    elif row.code == 'P' or row.code == 'I': continue
      # df_pitch.at[i, 'swing'] = 0
      # df_pitch.at[i, 'result'] = ball

    # batter takes and its a strike
    elif row.code == 'C':
      pitch.at[i, 'swing'] = 0
      pitch.at[i, 'result'] = strike

    # batter takes but gets on base
    elif row.code == 'H': continue
      # not sure how to treat this, is it a walk? or a hit?
      # if you consider it a hit then you are saying swing at pitches that hit you

    # this contains all rows in which code does not exist and is NaN
    elif row.code != row.code: continue
      # because this is a trivial amount of data we will ignore it

    # batter swings and ball is in play
    elif row.type == 'X':

      # res compares the batter_id from pitch.csv to atbat.csv to see the outcome of the atbat
      res = row.event

      # batter is trying to get on base but gets out
      if (res == "Groundout" or res == "Flyout" or res == "Lineout" or res == "Forceout" or 
            res == "Pop Out" or res == "Grounded Into DP" or res == "Double Play" or res == "Fielders Choice Out" or
            res == "Bunt Pop Out" or res == "Bunt Lineout" or res == "Triple Play" or res == "Bunt Groundout" or
            res == "Runner Out" or res == "Fielders Choice" or res == "Strikeout - DP" or res == "Batter Interference"):
        pitch.at[i, 'swing'] = 1
        pitch.at[i, 'result'] = out

      # batter is trying to get on base and succeeds
      elif res == "Single" or res == "Double" or res == "Triple" or res == "Home Run":
        pitch.at[i, 'swing'] = 1
        pitch.at[i, 'result'] = hit

      # batter is NOT trying to get on base
      elif res == "Sac Fly" or res == "Sac Bunt" or res == "Sac Fly DP" or res == "Sacrifice Bunt DP": continue
        # df_pitch.at[i, 'swing'] = 1
        # df_pitch.at[i, 'result'] = out

      # batter should be out but fielders make an error
      elif res == "Field Error" or res == "Catcher Interference": continue
        # df_pitch.at[i, 'swing'] = 1
        # df_pitch.at[i, 'result'] = hit

      # debugging results we do not take into account so we can assign them accordingly
      else: print(res)

/project/engineering/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/project/engineering/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0
10000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390000
1400000
1410000
1420000
1430000
1440000
145

In [50]:
# filter pitches where batter swung
filt = pitch['swing'] != -1
pitch = pitch.loc[filt]

In [52]:
def create_cutoffs():
  # Defining All Pitch's Obvious Zone Cutoffs
  FF_cutoffs = { '9a' : (-1.2, 1.4), '10a' : (1.5), '12a' : (-1.5), '13a' : (1.4), '15a' : (-1.25) }
  SL_cutoffs = { '10a' : (1.2), '12a' : (-1.4, 0.6), '13a' : (1.5, 0.6), 
                '14a' : (-1.6, -2.1), '15a' : (-2.1), '16a' : (1.5, -2.2) }
  CU_cutoffs = { '12a' : (-1.4, 0.5), '13a' : (1.5, 0.5), '14a' : (-1.3, -2.1), '15a' : (-2.4), '16a' : (1.3, -2.0) }
  CH_cutoffs = { '12a' : (-1.4, 0.9), '13a' : (1.5, 0.9), '14a' : (-1.4, -1.8), '15a' : (-1.8), '16a' : (1.4, -1.9) }
  FT_cutoffs = { '9a' : (-1.3, 1.4), '10a' : (1.2), '12a' : (-1.5), '13a' : (1.2), '14a' : (-1.2, -1.4), '15a' : (-1.5) }
  FC_cutoffs = { '10a' : (1.2), '12a' : (-1.5, 0.8), '13a' : (1.6, 0.8), 
                '14a' : (-1.5, -1.8), '15a' : (-1.9), '16a' : (1.5, -1.8) }
  SI_cutoffs = { '10a' : (1.2), '12a' : (-1.6), '13a' : (1.2), '14a' : (-1.5, -1.7), '15a' : (-1.7) }

  # Creating a Comprehensive List of Cutoffs 
  all_cutoffs = {
      "FF" : FF_cutoffs,
      "SL" : SL_cutoffs,
      "CU" : CU_cutoffs,
      "CH" : CH_cutoffs,
      "FT" : FT_cutoffs,
      "FC" : FC_cutoffs,
      "SI" : SI_cutoffs,
  }
  return all_cutoffs

cutoffs = create_cutoffs()

In [53]:
# Given px, pz coordinates we return the zone the pitch was in 
# pitch_cutoffs represent location where a zone changes from non-obvious to obvious
def SI_return_zone(px, pz, SI_cutoffs):
  if px < left_x:
    if pz > top_y: return '9b'
    elif pz > bot_y:
      if px > SI_cutoffs['12a']: return '12a'
      else: return '12b'
    else: 
      if px > SI_cutoffs['14a'][0] and pz > SI_cutoffs['14a'][1]: return '14a'
      else: return '14b'
      
  elif px < mid_left_x:
    if pz > SI_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a' 
    elif pz > SI_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < mid_right_x:
    if pz > SI_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a' 
    elif pz > SI_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < right_x:
    if pz > SI_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    elif pz > SI_cutoffs['15a']: return '15a'
    else: return '15b'

  else:
    if pz > top_y: return '11b'
    elif pz > bot_y: 
      if px < SI_cutoffs['13a']: return '13a'
      else: return '13b'
    else: return '16b'

  return '-1'


# Given px, pz coordinates we return the zone the pitch was in 
# pitch_cutoffs represent location where a zone changes from non-obvious to obvious
def FC_return_zone(px, pz, FC_cutoffs):
  if px < left_x:
    if pz > top_y: return '9b'
    elif pz > bot_y:
      if px > FC_cutoffs['12a'][0] and pz < FC_cutoffs['12a'][1]: return '12a'
      else: return '12b'
    else: 
      if px > FC_cutoffs['14a'][0] and pz > FC_cutoffs['14a'][1]: return '14a'
      else: return '14b'

  elif px < mid_left_x:
    if pz > FC_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a' 
    elif pz > FC_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < mid_right_x:
    if pz > FC_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a' 
    elif pz > FC_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < right_x:
    if pz > FC_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    elif pz > FC_cutoffs['15a']: return '15a'
    else: return '15b'

  else:
    if pz > top_y: return '11b'
    elif pz > bot_y:
      if px < FC_cutoffs['13a'][0] and pz < FC_cutoffs['13a'][1]: return '13a'
      else: return '13b'
    else:
      if px < FC_cutoffs['16a'][0] and pz > FC_cutoffs['16a'][1]: return '16a'
      else: return '16b'

  return '-1'

# Given px, pz coordinates we return the zone the pitch was in 
# pitch_cutoffs represent location where a zone changes from non-obvious to obvious
def FT_return_zone(px, pz, FT_cutoffs):
  if px < left_x:
    if pz > top_y:
      if px > FT_cutoffs['9a'][0] and pz < FT_cutoffs['9a'][1]: return '9a'
      else: return '9b'
    elif pz > bot_y:
      if px > FT_cutoffs['12a']: return '12a'
      else: return '12b'
    else: 
      if px > FT_cutoffs['14a'][0] and pz > FT_cutoffs['14a'][1]: return '14a'
      else: return '14b'
      
  elif px < mid_left_x:
    if pz > FT_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a' 
    elif pz > FT_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < mid_right_x:
    if pz > FT_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a' 
    elif pz > FT_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < right_x:
    if pz > FT_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    elif pz > FT_cutoffs['15a']: return '15a'
    else: return '15b'

  else:
    if pz > top_y: return '11b'
    elif pz > bot_y: 
      if px < FT_cutoffs['13a']: return '13a'
      else: return '13b'
    else: return '16b'

  return '-1'

# Given px, pz coordinates we return the zone the pitch was in 
# pitch_cutoffs represent location where a zone changes from non-obvious to obvious
def CH_return_zone(px, pz, CH_cutoffs):
  if px < left_x:
    if pz > top_y: return '9b'
    elif pz > bot_y:
      if px > CH_cutoffs['12a'][0] and pz < CH_cutoffs['12a'][1]: return '12a'
      else: return '12b'
    else: 
      if px > CH_cutoffs['14a'][0] and pz > CH_cutoffs['14a'][1]: return '14a'
      else: return '14b'

  elif px < mid_left_x:
    if pz > top_y: return '10b'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a' 
    elif pz > CH_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < mid_right_x:
    if pz > top_y: return '10b'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a' 
    elif pz > CH_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < right_x:
    if pz > top_y: return '10b'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    elif pz > CH_cutoffs['15a']: return '15a'
    else: return '15b'

  else:
    if pz > top_y: return '11b'
    elif pz > bot_y:
      if px < CH_cutoffs['13a'][0] and pz < CH_cutoffs['13a'][1]: return '13a'
      else: return '13b'
    else:
      if px < CH_cutoffs['16a'][0] and pz > CH_cutoffs['16a'][1]: return '16a'
      else: return '16b'

  return '-1'

# Given px, pz coordinates we return the zone the pitch was in 
# pitch_cutoffs represent location where a zone changes from non-obvious to obvious
def CU_return_zone(px, pz, CU_cutoffs):
  if px < left_x:
    if pz > top_y: return '9b'
    elif pz > bot_y:
      if px > CU_cutoffs['12a'][0] and pz < CU_cutoffs['12a'][1]: return '12a'
      else: return '12b'
    else: 
      if px > CU_cutoffs['14a'][0] and pz > CU_cutoffs['14a'][1]: return '14a'
      else: return '14b'

  elif px < mid_left_x:
    if pz > top_y: return '10b'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a' 
    elif pz > CU_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < mid_right_x:
    if pz > top_y: return '10b'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a' 
    elif pz > CU_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < right_x:
    if pz > top_y: return '10b'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    elif pz > CU_cutoffs['15a']: return '15a'
    else: return '15b'

  else:
    if pz > top_y: return '11b'
    elif pz > bot_y:
      if px < CU_cutoffs['13a'][0] and pz < CU_cutoffs['13a'][1]: return '13a'
      else: return '13b'
    else:
      if px < CU_cutoffs['16a'][0] and pz > CU_cutoffs['16a'][1]: return '16a'
      else: return '16b'

  return '-1'

# Given px, pz coordinates we return the zone the pitch was in 
# pitch_cutoffs represent location where a zone changes from non-obvious to obvious
def SL_return_zone(px, pz, SL_cutoffs):
  if px < left_x:
    if pz > top_y: return '9b'
    elif pz > bot_y:
      if px > SL_cutoffs['12a'][0] and pz < SL_cutoffs['12a'][1]: return '12a'
      else: return '12b'
    else: 
      if px > SL_cutoffs['14a'][0] and pz > SL_cutoffs['14a'][1]: return '14a'
      else: return '14b'

  elif px < mid_left_x:
    if pz > SL_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a' 
    elif pz > SL_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < mid_right_x:
    if pz > SL_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a' 
    elif pz > SL_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < right_x:
    if pz > SL_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    elif pz > SL_cutoffs['15a']: return '15a'
    else: return '15b'

  else:
    if pz > top_y: return '11b'
    elif pz > bot_y:
      if px < SL_cutoffs['13a'][0] and pz < SL_cutoffs['13a'][1]: return '13a'
      else: return '13b'
    else:
      if px < SL_cutoffs['16a'][0] and pz > SL_cutoffs['16a'][1]: return '16a'
      else: return '16b'

  return '-1'

# Given px, pz coordinates we return the zone the pitch was in 
# pitch_cutoffs represent location where a zone changes from non-obvious to obvious
def FF_return_zone(px, pz, FF_cutoffs):
  if px < left_x:
    if pz > top_y:
      if px > FF_cutoffs['9a'][0] and pz < FF_cutoffs['9a'][1]: return '9a'
      else: return '9b'
    elif pz > bot_y:
      if px > FF_cutoffs['12a']: return '12a'
      else: return '12b'
    else: return '14b'

  elif px < mid_left_x:
    if pz > FF_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a' 
    elif pz > FF_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < mid_right_x:
    if pz > FF_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a' 
    elif pz > FF_cutoffs['15a']: return '15a'
    else: return '15b'

  elif px < right_x:
    if pz > FF_cutoffs['10a']: return '10b'
    elif pz > top_y: return '10a'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    elif pz > FF_cutoffs['15a']: return '15a'
    else: return '15b'

  else:
    if pz > top_y: return '11b'
    elif pz > bot_y: 
      if px < FF_cutoffs['13a']: return '13a'
      else: return '13b'
    else: return '16b'

  return '-1'
def general_return_zone(px, pz):
  
  if px < left_x:
    if pz > top_y: return '9a'
    elif pz > bot_y: return '12a'
    else: return '14a'

  elif px < mid_left_x:
    if pz > top_y: return '10a'
    elif pz > mid_top_y: return '0a'
    elif pz > mid_bot_y: return '3a'
    elif pz > bot_y: return '6a'
    else: return '15a'

  elif px < mid_right_x:
    if pz > top_y: return '10a'
    elif pz > mid_top_y: return '1a'
    elif pz > mid_bot_y: return '4a'
    elif pz > bot_y: return '7a'
    else: return '15a'

  elif px < right_x:
    if pz > top_y: return '10a'
    elif pz > mid_top_y: return '2a'
    elif pz > mid_bot_y: return '5a'
    elif pz > bot_y: return '8a'
    else: return '15a'

  else:
    if pz > top_y: return '11a'
    elif pz > bot_y: return '13a'
    else: return '16a'

  return '-1'

# Creating our own Zone
# +---+-----------+---+
# |9  |     10    | 11|  
# |---+-----------+---|
# |   | 0 | 1 | 2 |   |
# |   |---+---+---|   |
# |12 | 3 | 4 | 5 | 13|
# |   |---+---+---|   |
# |   | 6 | 7 | 8 |   |
# |---+-----------+---|
# |14 |     15    | 16|
# +---+-----------+---+

# Splitting Obvious Zones for each Pitch:

# FF                  (not obvious with x,y coordinate ranges)
# 9: (-.831 to -1.2), (1.074 to 1.4)
# 10: (-.831 to .831), (1.074 to 1.5)
# 11: obvious
# 12: (-.831 to -1.5), (-1.074 to 1.074)
# 13: (.831 to 1.4), (-1.074 to 1.074)
# 14: obvious
# 15: (-.831 to .831), (-1.25 to -1.074)
# 16: obvious

# SL                  (not obvious with x,y coordinate ranges)
# 9: obvious
# 10: (-.831 to .831), (1.074 to 1.2)
# 11: obvious
# 12: (-.831 to -1.4), (-1.074 to 0.6)
# 13: (.831 to 1.5), (-1.074 to 0.6)
# 14: (-.831 to -1.6), (-2.1 to -1.074)
# 15: (-.831 to .831), (-2.1 to -1.074)
# 16: (.831 to 1.5), (-2.2 to -1.074)

# CU                  (not obvious with x,y coordinate ranges)
# 9: obvious
# 10: obvious
# 11: obvious
# 12: (-.831 to -1.4), (-1.074 to 0.5)
# 13: (.831 to 1.5), (-1.074 to 0.5)
# 14: (-.831 to -1.3), (-2.1 to -1.074)
# 15: (-.831 to .831), (-2.4 to -1.074)
# 16: (.831 to 1.3), (-2.0 to -1.074)

# CH                  (not obvious with x,y coordinate ranges)
# 9: obvious
# 10: obvious
# 11: obvious
# 12: (-.831 to -1.4), (-1.074 to 0.9)
# 13: (.831 to 1.5), (-1.074 to 0.9)
# 14: (-.831 to -1.4), (-1.8 to -1.074)
# 15: (-.831 to .831), (-1.8 to -1.074)
# 16: (.831 to 1.4), (-1.9 to -1.074)

# FT                  (not obvious with x,y coordinate ranges)
# 9: (-.831 to -1.3), (1.074 to 1.4)
# 10: (-.831 to .831), (1.074 to 1.2)
# 11: obvious
# 12: (-.831 to -1.5), (-1.074 to 1.074)
# 13: (.831 to 1.2), (-1.074 to 1.074)
# 14: (-.831 to -1.2), (-1.4 to -1.074)
# 15: (-.831 to .831), (-1.5 to -1.074)
# 16: obvious

# FC                  (not obvious with x,y coordinate ranges) SIMILAR TO SLIDER
# 9: obvious 
# 10: (-.831 to .831), (1.074 to 1.2) 
# 11: obvious 
# 12: (-.831 to -1.5), (-1.074 to 0.8) 
# 13: (.831 to 1.6), (-1.074 to 0.8)
# 14: (-.831 to -1.5), (-1.8 to -1.074) 
# 15: (-.831 to .831), (-1.9 to -1.074) 
# 16: (.831 to 1.5), (-1.8 to -1.074) 

# SI                  (not obvious with x,y coordinate ranges) SIMILAR TO TWO-SEAM
# 9: obvious 
# 10: (-.831 to .831), (1.074 to 1.2) 
# 11: obvious 
# 12: (-.831 to -1.6), (-1.074 to 1.074) 
# 13: (.831 to 1.2), (-1.074 to 1.074)
# 14: (-.831 to -1.5), (-1.7 to -1.074) 
# 15: (-.831 to .831), (-1.7 to -1.074) 
# 16: obvious

left_x = -.831
mid_left_x = -.277
mid_right_x = .277
right_x = .831

top_y = 1.074
mid_top_y = .358
mid_bot_y = -.358
bot_y = -1.074

In [54]:
# Adds pitch zone
def add_pitch_zone(df, cutoffs, rescale_z=2.599):
  df['zones'] = '-1'
  for i, row in df.iterrows():
    if row.pitch_type == "FF":
      df.at[i, 'zones'] = FF_return_zone(row.px, row.py-rescale_z, cutoffs[row.pitch_type])
    elif row.pitch_type == "SL":
      df.at[i, 'zones'] = SL_return_zone(row.px, row.py-rescale_z, cutoffs[row.pitch_type])
    elif row.pitch_type == "CU":
      df.at[i, 'zones'] = CU_return_zone(row.px, row.py-rescale_z, cutoffs[row.pitch_type])
    elif row.pitch_type == "CH":
      df.at[i, 'zones'] = CH_return_zone(row.px, row.py-rescale_z, cutoffs[row.pitch_type])
    elif row.pitch_type == "FT":
      df.at[i, 'zones'] = FT_return_zone(row.px, row.py-rescale_z, cutoffs[row.pitch_type])
    elif row.pitch_type == "FC":
      df.at[i, 'zones'] = FC_return_zone(row.px, row.py-rescale_z, cutoffs[row.pitch_type])
    elif row.pitch_type == "SI":
      df.at[i, 'zones'] = SI_return_zone(row.px, row.py-rescale_z, cutoffs[row.pitch_type])
    else:
      df.at[i, 'zones'] = general_return_zone(row.px, row.py-rescale_z)
  return df
pitch = add_pitch_zone(pitch, cutoffs)

/project/engineering/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [55]:
# save pitch data
pitch.to_csv("all_pitch.csv")

In [67]:
# filter at bat columns
at_bat_keep_cols = ["ab_id","batter_id","event","pitcher_id"]
old_atbat_merge = old_atbat[at_bat_keep_cols]
total_atbat = old_atbat_merge

In [70]:
# save atbat data
total_atbat.to_csv("all_atbat.csv")